In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
"/content/drive/MyDrive/deep/images_001.zip"

'/content/drive/MyDrive/deep/images_001.zip'

In [51]:
import zipfile

# zip_pathをzipファイルへの正しいパスに修正します
# The zip_path should point to the zip file itself, not a directory inside it.
zip_path = "/content/drive/MyDrive/deep/images_001.zip"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
  # 必要であれば、extractallの引数で解凍先のディレクトリを指定します
  # Specify the extraction directory if needed
  zip_ref.extractall("archive")

# print は引数がないと何も表示しません。何か表示したい場合は引数を追加してください。
# If you want to print something, add arguments to the print function.
# print("Zip file extracted successfully.")

In [56]:
import os
print(os.listdir("archive")) # List the contents of the extracted directory

['images_001', '__MACOSX']


In [57]:
import pandas as pd

csv_path = "/content/drive/MyDrive/deep/Data_Entry_2017.csv"

df = pd.read_csv(csv_path)

df.head

<bound method NDFrame.head of              Image Index          Finding Labels  Follow-up #  Patient ID  \
0       00000001_000.png            Cardiomegaly            0           1   
1       00000001_001.png  Cardiomegaly|Emphysema            1           1   
2       00000001_002.png   Cardiomegaly|Effusion            2           1   
3       00000002_000.png              No Finding            0           2   
4       00000003_000.png                  Hernia            0           3   
...                  ...                     ...          ...         ...   
112115  00030801_001.png          Mass|Pneumonia            1       30801   
112116  00030802_000.png              No Finding            0       30802   
112117  00030803_000.png              No Finding            0       30803   
112118  00030804_000.png              No Finding            0       30804   
112119  00030805_000.png              No Finding            0       30805   

        Patient Age Patient Gender View Position  OriginalImage[Width  \
0                58              M            PA                 2682   
1                58              M            PA                 2894   
2                58              M            PA                 2500   
3                81              M            PA                 2500   
4                81              F            PA                 2582   
...             ...            ...           ...                  ...   
112115           39              M            PA                 2048   
112116           29              M            PA                 2048   
112117           42              F            PA                 2048   
112118           30              F            PA                 2048   
112119           27              M            PA                 2048   

        Height]  OriginalImagePixelSpacing[x     y]  Unnamed: 11  
0          2749                        0.143  0.143          NaN  
1          2729                        0.143  0.143          NaN  
2          2048                        0.168  0.168          NaN  
3          2048                        0.171  0.171          NaN  
4          2991                        0.143  0.143          NaN  
...         ...                          ...    ...          ...  
112115     2500                        0.168  0.168          NaN  
112116     2500                        0.168  0.168          NaN  
112117     2500                        0.168  0.168          NaN  
112118     2500                        0.168  0.168          NaN  
112119     2500                        0.171  0.171          NaN  

[112120 rows x 12 columns]>

In [58]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/deep/Data_Entry_2017.csv")

# Massを含むかどうかで癌(1)　or 癌じゃない(0)のラベルを作る
df["Cancer"] = df["Finding Labels"].apply(lambda x: 1 if "Mass" in x else 0)

print(df["Cancer"].value_counts())

df[["Image Index", "Finding Labels", "Cancer"]].head()

Cancer
0    106338
1      5782
Name: count, dtype: int64


,Image Index,Finding Labels,Cancer
0,00000001_000.png,Cardiomegaly,0
1,00000001_001.png,Cardiomegaly|Emphysema,0
2,00000001_002.png,Cardiomegaly|Effusion,0
3,00000002_000.png,No Finding,0
4,00000003_000.png,Hernia,0


In [59]:

import os
from tqdm import tqdm
import cv2
import numpy as np

image_dir = "images_001/images"

images = []
labels = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
  filename = row["Image Index"]
  label = row["Cancer"]
  image_path = os.path.join(image_dir, filename)



  if os.path.exists(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128, 128))
    images.append(img)
    labels.append(label)


X = np.array(images)
y = np.array(labels)


print("画像データの形:", len(X))
print("ラベルの形:", len(y))

100%|██████████| 112120/112120 [00:08<00:00, 12730.24it/s]

画像データの形: 0
ラベルの形: 0


In [63]:
import os
from tqdm import tqdm
import cv2
import numpy as np

# image_dirを修正して、解凍先のディレクトリ内の画像ディレクトリを指すようにします
# Fix image_dir to point to the image directory within the extraction directory
image_dir = "archive/images_001/images"

images = []
labels = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
  filename = row["Image Index"]
  label = row["Cancer"]
  image_path = os.path.join(image_dir, filename)



  if os.path.exists(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128, 128))
    images.append(img)
    labels.append(label)


X = np.array(images)
y = np.array(labels)


print("画像データの形:", len(X))
print("ラベルの形:", len(y))



100%|██████████| 112120/112120 [01:25<00:00, 1313.37it/s] 

画像データの形: 4999
ラベルの形: 4999


In [64]:
print("Xの中身の数:",len(X))
print("ラベルの数:", len(y))

Xの中身の数: 4999
ラベルの数: 4999


In [65]:
from sklearn.model_selection import train_test_split

X = X / 255.0

X = X.reshape(-1, 128, 128, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


print("学習データ数:", len(X_train))
print("テストデータ数:", len(X_test))

学習データ数: 3999
テストデータ数: 1000


In [70]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32,(3,3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,705,345 (14.13 MB)

 Trainable params: 3,705,345 (14.13 MB)

 Non-trainable params: 0 (0.00 B)

In [73]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32,
                    validation_data=(X_test, y_test))

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 101s 796ms/step - accuracy: 0.9284 - loss: 0.2042 - val_accuracy: 0.9680 - val_loss: 0.1467
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 143s 798ms/step - accuracy: 0.9686 - loss: 0.1453 - val_accuracy: 0.9680 - val_loss: 0.1516
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 141s 797ms/step - accuracy: 0.9618 - loss: 0.1676 - val_accuracy: 0.9680 - val_loss: 0.1522
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 795ms/step - accuracy: 0.9648 - loss: 0.1594 - val_accuracy: 0.9680 - val_loss: 0.1628
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 143s 807ms/step - accuracy: 0.9695 - loss: 0.1337 - val_accuracy: 0.9680 - val_loss: 0.1615


In [74]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("テスト精度:", test_acc)

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 186ms/step - accuracy: 0.9680 - loss: 0.1653
テスト精度: 0.9679999947547913
